In [12]:
import numpy as np
import pandas as pd

In [13]:
df = pd.read_csv("XAU_1m_data.csv", sep=";")
df

,Date,Open,High,Low,Close,Volume
0,2004.06.11 07:18,384.00,384.10,384.00,384.00,3
1,2004.06.11 07:23,384.10,384.10,384.00,384.00,2
2,2004.06.11 07:24,383.80,383.80,383.80,383.80,1
3,2004.06.11 07:25,383.80,384.30,383.80,384.30,3
4,2004.06.11 07:27,383.80,383.80,383.80,383.80,1
...,...,...,...,...,...,...
6695755,2025.12.01 04:15,4248.22,4248.73,4247.18,4247.58,525
6695756,2025.12.01 04:16,4247.75,4248.30,4246.45,4246.59,391
6695757,2025.12.01 04:17,4246.57,4248.02,4246.13,4246.26,339
6695758,2025.12.01 04:18,4246.22,4247.23,4246.09,4246.44,299


In [14]:
df.isna().sum()

Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [15]:
df.duplicated().sum()

0

In [16]:
df.describe(include='all')

,Date,Open,High,Low,Close,Volume
count,6695760,6.695760e+06,6.695760e+06,6.695760e+06,6.695760e+06,6.695760e+06
unique,6695760,NaN,NaN,NaN,NaN,NaN
top,2004.06.11 07:18,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN
mean,NaN,1.457690e+03,1.457906e+03,1.457471e+03,1.457689e+03,6.307870e+01
std,NaN,5.616410e+02,5.617312e+02,5.615506e+02,5.616422e+02,7.978339e+01
min,NaN,3.811000e+02,3.811000e+02,3.811000e+02,3.811000e+02,1.000000e+00
25%,NaN,1.179090e+03,1.179260e+03,1.178930e+03,1.179090e+03,1.600000e+01
50%,NaN,1.324620e+03,1.324800e+03,1.324450e+03,1.324620e+03,4.400000e+01
75%,NaN,1.772140e+03,1.772410e+03,1.771890e+03,1.772140e+03,8.200000e+01


In [17]:
df.describe().T.style.background_gradient(axis=0)

,count,mean,std,min,25%,50%,75%,max
Open,6695760.000000,1457.689686,561.640971,381.100000,1179.090000,1324.620000,1772.140000,4254.980000
High,6695760.000000,1457.906337,561.731238,381.100000,1179.260000,1324.800000,1772.410000,4256.340000
Low,6695760.000000,1457.470983,561.550588,381.100000,1178.930000,1324.450000,1771.890000,4252.710000
Close,6695760.000000,1457.689445,561.642211,381.100000,1179.090000,1324.620000,1772.140000,4255.040000
Volume,6695760.000000,63.078697,79.783390,1.000000,16.000000,44.000000,82.000000,5959.000000


In [ ]:
# ================================================================================
# GENERAL EDA - XAUUSD (2004-2024)
# Complete Step-by-Step Analysis Without Functions or Abstractions
# Purpose: Deep understanding of data before modeling
# ================================================================================

# ================================================================================
# SECTION 0: LIBRARY IMPORTS
# ================================================================================
# We import libraries upfront. These are tools we'll use throughout the notebook.
# Each import is explained so you understand what it does.

import pandas as pd                    # For data manipulation and analysis
import numpy as np                     # For numerical operations and array handling
import matplotlib.pyplot as plt        # For creating visualizations (plots, charts)
import seaborn as sns                  # For statistical visualizations (prettier than matplotlib)
from datetime import datetime, timedelta  # For working with dates and time
import warnings                        # To suppress warning messages
warnings.filterwarnings('ignore')      # Suppress warnings for cleaner output

# Statistical libraries for analysis
from scipy import stats                # For statistical tests (mean, std, distributions)
from scipy.stats import skew, kurtosis  # For measuring distribution shape
from statsmodels.tsa.stattools import adfuller  # For stationarity testing
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf  # For autocorrelation

# Set display options for better readability
pd.set_option('display.max_columns', None)  # Show all columns when printing dataframes
pd.set_option('display.max_rows', 100)      # Show max 100 rows when printing
pd.set_option('display.float_format', lambda x: f'{x:.4f}')  # Show 4 decimal places

# Set visual style for plots
plt.style.use('seaborn-v0_8-darkgrid')  # Dark background with grid
sns.set_palette("husl")                  # Color palette for plots
plt.rcParams['figure.figsize'] = (14, 6) # Default figure size

print("=" * 80)
print("GENERAL EDA - XAUUSD (2004-2024)")
print("Step-by-Step Analysis Without Functions")
print("=" * 80)

# ================================================================================
# SECTION 1: LOAD DATA
# ================================================================================
# Load the CSV file from MT4/MT5 export
# The file should be in the same directory as this notebook or specify the path

print("\n[SECTION 1] LOADING DATA FROM CSV...")

# Read the CSV file
# We use pd.read_csv() to load the data into a DataFrame
# A DataFrame is like a spreadsheet with rows and columns
df = pd.read_csv('xauusd_2004_2024.csv')

print(f"✓ Data loaded successfully!")
print(f"  Total rows: {len(df):,}")
print(f"  Total columns: {df.shape[1]}")
print(f"  Columns: {list(df.columns)}")

# Show first few rows to understand the structure
print("\nFirst 5 rows of data:")
print(df.head())

# Show data types of each column
print("\nData types:")
print(df.dtypes)

# ================================================================================
# SECTION 2: PARSE DATETIME AND DETECT TIMEZONE
# ================================================================================
# MT4 exports dates in format "2004.06.11 07:18" (YYYY.MM.DD HH:MM)
# We need to convert this text into proper datetime objects that Python understands
# Then we check if the times are UTC or broker-adjusted

print("\n[SECTION 2] PARSING DATETIME AND CHECKING TIMEZONE...")

# Convert the Date column from text to datetime objects
# The format parameter tells pandas how to interpret the date string
# %Y = 4-digit year (2004)
# %m = 2-digit month (06)
# %d = 2-digit day (11)
# %H = 2-digit hour (07)
# %M = 2-digit minute (18)
df['Date'] = pd.to_datetime(df['Date'], format='%Y.%m.%d %H:%M')

print(f"✓ DateTime parsed successfully!")
print(f"  First timestamp: {df['Date'].iloc[0]}")
print(f"  Last timestamp: {df['Date'].iloc[-1]}")

# Set Date as index (this makes it the row identifier)
# This is useful for time-series analysis
df.set_index('Date', inplace=True)
df.index.name = 'DateTime'

print(f"\nDataFrame index is now DateTime (first 5):")
print(df.index[:5])

# Now check timezone: Is this UTC or broker-adjusted time?
# XAUUSD trades 24 hours on forex (Sunday 5PM EST to Friday 5PM EST)
# If we see trading activity in all 24 hours, it's likely UTC
# If we see gaps at specific hours, it might be broker time

print("\n--- TIMEZONE DETECTION ---")
print("\nAnalyzing which hours have trading activity...")

# Extract the hour from each timestamp
# For example: 2004-06-11 07:18 → hour = 7
hours_in_data = df.index.hour

# Count how many times each hour appears
hour_counts = hours_in_data.value_counts().sort_index()

print("\nHour distribution (showing how many candles in each hour):")
for hour in range(24):
    count = hour_counts.get(hour, 0)
    # Create a visual bar to show distribution
    bar = '█' * (count // 1000)  # Each block = ~1000 candles
    print(f"  Hour {hour:02d}:00 - {count:8,} candles {bar}")

# Check if we have activity in all 24 hours
unique_hours = len(hour_counts)
print(f"\nUnique hours with data: {unique_hours}/24")

# XAUUSD typically trades all 24 hours on forex
# If we see all 24 hours, it's UTC-based
if unique_hours == 24:
    print("✓ Likely UTC timezone (trading active all 24 hours)")
    detected_timezone = "UTC"
else:
    print("⚠ Possible broker adjustment or limited trading hours")
    detected_timezone = "Broker-adjusted"

print(f"\nTimezone: {detected_timezone}")

# ================================================================================
# SECTION 3: CHECK BASIC DATA STRUCTURE
# ================================================================================
# Before we analyze the data, we need to make sure the data is valid
# We check for missing values, duplicates, and data integrity

print("\n[SECTION 3] CHECKING DATA STRUCTURE AND INTEGRITY...")

# Check for missing values (NaN, None, empty cells)
print("\nMissing values per column:")
missing_counts = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df)) * 100

for column in df.columns:
    missing = missing_counts[column]
    pct = missing_pct[column]
    if missing > 0:
        print(f"  {column}: {missing:,} missing ({pct:.4f}%)")
    else:
        print(f"  {column}: 0 missing (0.00%) ✓")

# If Close has missing values, we need to fill them
# We use interpolation (linear interpolation fills gaps smoothly)
if df['Close'].isnull().sum() > 0:
    print(f"\n⚠ Found {df['Close'].isnull().sum()} missing Close values")
    print("  Applying linear interpolation to fill missing values...")
    df['Close'] = df['Close'].interpolate(method='linear')
    print("  ✓ Interpolation complete")

# Check for duplicate timestamps
# Each timestamp should appear only once (one 1-minute candle per minute)
print(f"\nDuplicate timestamps: {df.index.duplicated().sum()}")

if df.index.duplicated().sum() > 0:
    print(f"  Removing {df.index.duplicated().sum()} duplicate rows...")
    df = df[~df.index.duplicated(keep='first')]
    print(f"  ✓ Duplicates removed. New shape: {df.shape[0]:,} rows")

# Check data integrity: OHLC relationships
# High must be >= Open and Close (logically, high is the highest price)
# Low must be <= Open and Close (low is the lowest price)
# High must be >= Low (always)

print("\nOHLC Integrity Checks:")

# Check 1: High >= Open AND High >= Close
high_check = (df['High'] >= df['Open']) & (df['High'] >= df['Close'])
high_valid = high_check.sum()
print(f"  High >= Open & Close: {high_valid:,}/{len(df):,} ({(high_valid/len(df)*100):.2f}%) ✓")

# Check 2: Low <= Open AND Low <= Close
low_check = (df['Low'] <= df['Open']) & (df['Low'] <= df['Close'])
low_valid = low_check.sum()
print(f"  Low <= Open & Close: {low_valid:,}/{len(df):,} ({(low_valid/len(df)*100):.2f}%) ✓")

# Check 3: High >= Low (fundamental check)
hl_check = df['High'] >= df['Low']
hl_valid = hl_check.sum()
print(f"  High >= Low: {hl_valid:,}/{len(df):,} ({(hl_valid/len(df)*100):.2f}%) ✓")

# Mark any rows with integrity issues for inspection
invalid_rows = (~high_check) | (~low_check) | (~hl_check)
num_invalid = invalid_rows.sum()

if num_invalid > 0:
    print(f"\n⚠ Warning: {num_invalid} rows have OHLC integrity issues")
    print("  These will be flagged but not removed (for investigation)")
else:
    print("\n✓ All OHLC relationships are valid!")

# ================================================================================
# SECTION 4: TIME-SERIES STRUCTURE - GAPS AND MISSING CANDLES
# ================================================================================
# Check if there are gaps in the time series
# Gaps would indicate missing candles (e.g., market closed on weekends)
# We expect gaps on weekends and holidays

print("\n[SECTION 4] ANALYZING TIME-SERIES GAPS...")

# Calculate the time difference between consecutive candles
# In a normal situation, this should be 1 minute
time_differences = df.index.to_series().diff()

# Find all gaps larger than 1 minute
# dropna() removes the first row (which has NaN difference)
gaps = time_differences[time_differences > timedelta(minutes=1)].dropna()

print(f"\nTime gaps detected: {len(gaps):,}")
print(f"  Average gap: {gaps.mean()}")
print(f"  Largest gap: {gaps.max()}")
print(f"  Smallest gap: {gaps.min()}")

# Analyze which days of the week have gaps
# Days: 0=Monday, 1=Tuesday, ..., 6=Sunday
gap_day_of_week = gaps.index.dayofweek
gap_distribution = gap_day_of_week.value_counts().sort_index()

day_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday',
             4: 'Friday', 5: 'Saturday', 6: 'Sunday'}

print(f"\nGaps by day of week:")
for day_num in range(7):
    count = gap_distribution.get(day_num, 0)
    print(f"  {day_names[day_num]}: {count:,} gaps")

print(f"\n✓ Gap analysis complete (expected: gaps on weekends)")

# ================================================================================
# SECTION 5: BASIC STATISTICS - UNIVARIATE ANALYSIS PART 1
# ================================================================================
# Now we dive into analyzing individual variables
# We start with basic statistics: mean, std, min, max, etc.

print("\n[SECTION 5] BASIC STATISTICS (UNIVARIATE ANALYSIS)...")

print("\n--- OPEN PRICES ---")
print(df['Open'].describe())

print("\n--- HIGH PRICES ---")
print(df['High'].describe())

print("\n--- LOW PRICES ---")
print(df['Low'].describe())

print("\n--- CLOSE PRICES ---")
print(df['Close'].describe())

print("\n--- VOLUME ---")
print(df['Volume'].describe())

# Calculate additional statistics manually to understand them better
print("\n--- ADDITIONAL STATISTICS FOR CLOSE PRICES ---")

close_prices = df['Close']

# Mean: average price
mean_close = close_prices.mean()
print(f"Mean (average): ${mean_close:.2f}")

# Median: middle value when sorted
median_close = close_prices.median()
print(f"Median (middle): ${median_close:.2f}")

# Mode: most common value
mode_close = close_prices.mode()[0] if len(close_prices.mode()) > 0 else None
print(f"Mode (most common): ${mode_close:.2f}" if mode_close else "Mode: Not available")

# Standard deviation: how spread out the prices are
std_close = close_prices.std()
print(f"Std Dev (spread): ${std_close:.2f}")

# Variance: square of standard deviation
var_close = close_prices.var()
print(f"Variance: {var_close:.2f}")

# Range: difference between max and min
range_close = close_prices.max() - close_prices.min()
print(f"Range (max - min): ${range_close:.2f}")

# Coefficient of Variation: std / mean (shows relative volatility)
cv_close = (std_close / mean_close) * 100
print(f"Coefficient of Variation: {cv_close:.2f}%")

# ================================================================================
# SECTION 6: CALCULATE RETURNS
# ================================================================================
# Returns are more important than prices for trading
# A return shows the percentage change from one period to the next
# Returns are stationary (mean-reverting), while prices are not

print("\n[SECTION 6] CALCULATING RETURNS...")

# Simple return: (Price_today - Price_yesterday) / Price_yesterday
# This shows the percentage change
df['Simple_Return'] = df['Close'].pct_change()

# Log return: ln(Price_today / Price_yesterday)
# This is preferred for financial data (more mathematically stable)
df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))

print(f"\nSimple Return statistics:")
print(f"  Mean: {df['Simple_Return'].mean():.6f}")
print(f"  Std Dev: {df['Simple_Return'].std():.6f}")
print(f"  Min: {df['Simple_Return'].min():.6f}")
print(f"  Max: {df['Simple_Return'].max():.6f}")

print(f"\nLog Return statistics:")
print(f"  Mean: {df['Log_Return'].mean():.6f}")
print(f"  Std Dev: {df['Log_Return'].std():.6f}")
print(f"  Min: {df['Log_Return'].min():.6f}")
print(f"  Max: {df['Log_Return'].max():.6f}")

# Calculate additional statistics for returns
print(f"\nReturn Distribution Analysis:")

# Skewness: is the distribution skewed to the left or right?
# Positive skew = tail on right (more extreme high values)
# Negative skew = tail on left (more extreme low values)
skewness = skew(df['Log_Return'].dropna())
print(f"  Skewness: {skewness:.4f} (negative = left tail, positive = right tail)")

# Kurtosis: how fat are the tails?
# Normal distribution = 0 (excess kurtosis)
# Fat tails (extreme events common) = positive kurtosis
# Thin tails = negative kurtosis
kurt = kurtosis(df['Log_Return'].dropna())
print(f"  Kurtosis: {kurt:.4f} (positive = fat tails, extreme events common)")

# ================================================================================
# SECTION 7: STATIONARITY TESTING
# ================================================================================
# Stationarity is important for time-series modeling
# Stationary series have constant mean and variance over time
# Non-stationary series trend upward or downward

print("\n[SECTION 7] STATIONARITY TESTING (ADF TEST)...")

# Test 1: Close prices (should be non-stationary)
print("\nTesting Close prices (raw prices)...")
adf_close = adfuller(df['Close'].dropna(), autolag='AIC')
print(f"  ADF Statistic: {adf_close[0]:.6f}")
print(f"  P-value: {adf_close[1]:.6f}")

if adf_close[1] < 0.05:
    print(f"  Result: STATIONARY (p < 0.05) - Reject null hypothesis")
else:
    print(f"  Result: NON-STATIONARY (p >= 0.05) - Fail to reject null hypothesis")

# Test 2: Log returns (should be stationary)
print("\nTesting Log Returns...")
adf_returns = adfuller(df['Log_Return'].dropna(), autolag='AIC')
print(f"  ADF Statistic: {adf_returns[0]:.6f}")
print(f"  P-value: {adf_returns[1]:.6f}")

if adf_returns[1] < 0.05:
    print(f"  Result: STATIONARY (p < 0.05) - Good for modeling!")
else:
    print(f"  Result: NON-STATIONARY (p >= 0.05) - Might need differencing")

print("\n✓ Interpretation:")
print("  - Close prices should be non-stationary (they trend over time)")
print("  - Returns should be stationary (mean-reverting property)")

# ================================================================================
# SECTION 8: CALCULATE ROLLING VOLATILITY
# ================================================================================
# Volatility is how much the price moves
# We calculate rolling volatility to see how it changes over time
# Higher volatility = bigger price swings = more risk and opportunity

print("\n[SECTION 8] CALCULATING ROLLING VOLATILITY...")

# Rolling standard deviation of returns (window = number of periods)
# 30-minute window: std of returns over last 30 minutes
df['Volatility_30m'] = df['Log_Return'].rolling(window=30).std()

# 60-minute window: std of returns over last 60 minutes
df['Volatility_60m'] = df['Log_Return'].rolling(window=60).std()

# 1440-minute window: std of returns over last 24 hours (daily volatility)
df['Volatility_1d'] = df['Log_Return'].rolling(window=1440).std()

print(f"\n30-minute volatility: {df['Volatility_30m'].describe()}")
print(f"\n60-minute volatility: {df['Volatility_60m'].describe()}")
print(f"\n1-day volatility: {df['Volatility_1d'].describe()}")

# Annualized volatility (if we want annual perspective)
# Volatility is typically expressed as annual standard deviation
# Daily volatility * sqrt(252 trading days) ≈ annual volatility
# Minute volatility * sqrt(525600 minutes in a year) ≈ annual volatility

annual_volatility = df['Volatility_1d'].mean() * np.sqrt(252)
print(f"\nAnnualized volatility (daily rolling): {annual_volatility:.4f} or {annual_volatility*100:.2f}%")

# ================================================================================
# SECTION 9: IDENTIFY MARKET REGIMES (VOLATILITY-BASED)
# ================================================================================
# Markets behave differently in different regimes
# High volatility regime: risky, fast movements
# Low volatility regime: stable, slow movements
# We'll classify each period into regimes based on volatility

print("\n[SECTION 9] IDENTIFYING MARKET REGIMES...")

# Use the 1-day rolling volatility to classify regimes
# We calculate percentiles to define regime thresholds
vol_1d = df['Volatility_1d'].dropna()

# Calculate percentiles
vol_33_percentile = vol_1d.quantile(0.33)  # Lower third
vol_67_percentile = vol_1d.quantile(0.67)  # Upper third

print(f"\nVolatility Percentiles (for regime definition):")
print(f"  33rd percentile (low vol boundary): {vol_33_percentile:.6f}")
print(f"  67th percentile (high vol boundary): {vol_67_percentile:.6f}")

# Classify each candle into a regime
# 0 = Low volatility
# 1 = Medium volatility
# 2 = High volatility
df['Regime'] = 1  # Default to medium

# Low volatility: below 33rd percentile
df.loc[df['Volatility_1d'] <= vol_33_percentile, 'Regime'] = 0

# High volatility: above 67th percentile
df.loc[df['Volatility_1d'] > vol_67_percentile, 'Regime'] = 2

print(f"\nRegime Distribution:")
regime_counts = df['Regime'].value_counts().sort_index()
for regime, count in regime_counts.items():
    pct = (count / len(df)) * 100
    regime_name = ['LOW', 'MEDIUM', 'HIGH'][regime]
    print(f"  Regime {regime} ({regime_name:6}): {count:8,} candles ({pct:5.2f}%)")

# ================================================================================
# SECTION 10: VOLUME ANALYSIS - CHECKING USEFULNESS
# ================================================================================
# Volume tells us how much trading happened
# In forex, volume is often just tick count, not true trading volume
# We need to check if volume provides useful information

print("\n[SECTION 10] VOLUME ANALYSIS...")

print(f"\nVolume Statistics:")
print(f"  Mean: {df['Volume'].mean():.2f}")
print(f"  Std Dev: {df['Volume'].std():.2f}")
print(f"  Min: {df['Volume'].min():.0f}")
print(f"  Max: {df['Volume'].max():.0f}")
print(f"  Median: {df['Volume'].median():.2f}")

# Check how many unique volume values we have
unique_volumes = df['Volume'].nunique()
print(f"\nUnique volume values: {unique_volumes}")

if unique_volumes < 100:
    print("⚠ Very limited unique volume values - likely tick volume")
    print("  Tick volume (count of trades) may have limited predictive power")
else:
    print("✓ Good variety of volume values")

# Calculate correlation between volume and price changes
# If volume doesn't correlate with price changes, it's not useful
price_changes = df['Close'].diff().abs()
volume_close_corr = df['Volume'].corr(price_changes)

print(f"\nVolume correlation with price changes: {volume_close_corr:.4f}")
if abs(volume_close_corr) < 0.1:
    print("  → Very weak correlation (volume not predictive of price moves)")
else:
    print("  → Moderate correlation (volume may be useful)")

# ================================================================================
# SECTION 11: CANDLE COLOR ANALYSIS
# ================================================================================
# Candle color shows direction: Green (Up) or Red (Down)
# We analyze the distribution of up/down days
# This is relevant to your 3-candle reversal strategy

print("\n[SECTION 11] CANDLE COLOR ANALYSIS (UP vs DOWN CANDLES)...")

# Green candle: Close > Open (bullish)
green_candles = (df['Close'] > df['Open']).sum()

# Red candle: Close < Open (bearish)
red_candles = (df['Close'] < df['Open']).sum()

# Unchanged: Close == Open (doji)
unchanged_candles = (df['Close'] == df['Open']).sum()

total_candles = len(df)

print(f"\nCandle Distribution:")
print(f"  Green (Close > Open): {green_candles:8,} ({green_candles/total_candles*100:.2f}%)")
print(f"  Red   (Close < Open): {red_candles:8,} ({red_candles/total_candles*100:.2f}%)")
print(f"  Doji  (Close = Open): {unchanged_candles:8,} ({unchanged_candles/total_candles*100:.2f}%)")
print(f"  Total: {total_candles:8,}")

# For your strategy, we need to understand how often we get 3 consecutive candles
# of the same color. We'll count this in Section 15 (baseline strategy)

print(f"\nUp/Down Ratio: {green_candles/red_candles:.4f}")
if green_candles > red_candles:
    print("  → More up candles than down (upward bias over 20 years)")
else:
    print("  → More down candles than up (downward bias)")

# ================================================================================
# SECTION 12: CRISIS PERIODS ANALYSIS
# ================================================================================
# Historical crises affected gold prices
# We'll analyze how XAUUSD behaved during known crisis periods

print("\n[SECTION 12] CRISIS PERIODS ANALYSIS...")

# Define major crisis periods
crises = {
    '2008 Financial Crisis': (datetime(2008, 9, 1), datetime(2008, 12, 31)),
    '2020 COVID-19 Crash': (datetime(2020, 2, 15), datetime(2020, 4, 30)),
    '2022 Fed Rate Hikes': (datetime(2022, 3, 1), datetime(2022, 12, 31)),
}

print("\nAnalyzing price behavior during crisis periods...\n")

for crisis_name, (start_date, end_date) in crises.items():
    # Select data for this crisis period
    crisis_data = df[(df.index >= start_date) & (df.index <= end_date)]
    
    if len(crisis_data) == 0:
        print(f"⚠ {crisis_name}: No data in range")
        continue
    
    # Calculate statistics for this period
    start_price = crisis_data['Close'].iloc[0]
    end_price = crisis_data['Close'].iloc[-1]
    min_price = crisis_data['Close'].min()
    max_price = crisis_data['Close'].max()
    avg_price = crisis_data['Close'].mean()
    
    # Calculate returns during crisis
    period_return = (end_price - start_price) / start_price * 100
    drawdown = (min_price - crisis_data['Close'].iloc[0]) / crisis_data['Close'].iloc[0] * 100
    
    # Volatility during crisis
    crisis_volatility = crisis_data['Log_Return'].std()
    
    print(f"{crisis_name}:")
    print(f"  Period: {start_date.date()} to {end_date.date()}")
    print(f"  Start Price: ${start_price:.2f}")
    print(f"  End Price: ${end_price:.2f}")
    print(f"  Period Return: {period_return:+.2f}%")
    print(f"  Max Drawdown: {drawdown:.2f}%")
    print(f"  Price Range: ${min_price:.2f} - ${max_price:.2f}")
    print(f"  Volatility: {crisis_volatility:.6f} (daily returns std)")
    print()

# ================================================================================
# SECTION 13: MAXIMUM DRAWDOWN ANALYSIS
# ================================================================================
# Drawdown is the peak-to-trough decline
# Maximum drawdown shows the worst loss experienced
# Important for risk management

print("\n[SECTION 13] MAXIMUM DRAWDOWN ANALYSIS...")

# Calculate cumulative maximum price at each point
cumulative_max = df['Close'].cummax()

# Calculate drawdown at each point
# Drawdown = (Current Price - Peak) / Peak
drawdown = (df['Close'] - cumulative_max) / cumulative_max * 100

# Find maximum drawdown
max_drawdown = drawdown.min()

# Find when the maximum drawdown occurred
max_drawdown_date = drawdown.idxmin()

print(f"\nMaximum Drawdown: {max_drawdown:.2f}%")
print(f"Occurred on: {max_drawdown_date}")
print(f"Price at that time: ${df.loc[max_drawdown_date, 'Close']:.2f}")

# Calculate average and rolling maximum drawdowns
avg_drawdown = drawdown[drawdown < 0].mean()
print(f"\nAverage Drawdown (when negative): {avg_drawdown:.2f}%")

# Find longest drawdown period
# A drawdown period is when price is below recent peak
drawdown_periods = (drawdown < 0).astype(int)
drawdown_changes = drawdown_periods.diff().fillna(0)
drawdown_starts = drawdown_changes[drawdown_changes == 1].index
drawdown_ends = drawdown_changes[drawdown_changes == -1].index

if len(drawdown_starts) > 0 and len(drawdown_ends) > 0:
    longest_duration = 0
    longest_start = None
    
    for start in drawdown_starts:
        matching_end = drawdown_ends[drawdown_ends > start]
        if len(matching_end) > 0:
            end = matching_end[0]
            duration = (end - start).total_seconds() / 3600  # Convert to hours
            if duration > longest_duration:
                longest_duration = duration
                longest_start = start
    
    print(f"Longest drawdown period: {longest_duration:.0f} hours")

# ================================================================================
# SECTION 14: SPREAD AND SLIPPAGE ESTIMATION
# ================================================================================
# Spreads and slippage are transaction costs that reduce profitability
# We estimate realistic spreads based on the data

print("\n[SECTION 14] SPREAD AND SLIPPAGE ANALYSIS...")

# Bid-Ask Spread estimation:
# We can estimate spread by looking at the High-Low range within a candle
# Typical XAUUSD spread from brokers is 2-3 pips

# Calculate average range per candle
df['Candle_Range'] = df['High'] - df['Low']
avg_range = df['Candle_Range'].mean()

print(f"\nAverage candle range: ${avg_range:.4f}")
print(f"  (This is not spread, but shows typical intracandle movement)")

# For XAUUSD, typical brokers have these spreads:
typical_spread = 0.30  # 30 cents on gold is ~3 pips (10 cents per pip)
print(f"\nTypical XAUUSD spreads from brokers:")
print(f"  Standard brokers: 2-4 pips (~$0.20-$0.40)")
print(f"  ECN brokers: 0.5-1.5 pips (~$0.05-$0.15)")

# Slippage is additional cost when executing large orders
# We'll assume 1-2 pips for this analysis
typical_slippage = 0.15  # 1.5 pips
total_cost_per_trade = (typical_spread + typical_slippage)

print(f"  Total cost per round-trip trade (entry + exit): ~${total_cost_per_trade:.2f}")
print(f"\n⚠ Important: These costs must be included in backtesting!")
print(f"  A strategy needs to make at least this much to be profitable")

# ================================================================================
# SECTION 15: BASELINE STRATEGY ANALYSIS - YOUR 3-CANDLE REVERSAL
# ================================================================================
# YOUR EXACT STRATEGY (Final Version):
# BUY: 3 consecutive GREEN candles → Enter at close of 3rd → SL at midpoint of 2nd
# SELL: 3 consecutive RED candles → Enter at close of 3rd → SL at midpoint of 2nd
# TP: Always 2× the SL distance
# EXIT: Only at SL or TP (candle wicks ignored)
# REQUIREMENT: Exactly 3 consecutive same-color candles (no partial patterns)

print("\n[SECTION 15] BASELINE STRATEGY ANALYSIS - YOUR 3-CANDLE PATTERN...")
print("\nStrategy Rules (FINAL):")
print("  BUY:  3 GREEN candles → Enter @ close of 3rd → SL @ midpoint of 2nd")
print("  SELL: 3 RED candles → Enter @ close of 3rd → SL @ midpoint of 2nd")
print("  TP: Entry ± 2×(SL distance)")
print("  EXIT: Only at SL or TP")
print("  REQUIREMENT: Exactly 3 consecutive same-color candles")

# Create candle color column
# GREEN = 1 (Close > Open), RED = 0 (Close < Open)
df['Candle_Color'] = (df['Close'] > df['Open']).astype(int)

signals = []  # Store all entry signals found

# Loop through data looking for exactly 3 consecutive same-color candles
# We start at index 2 because we need candles at i-2, i-1, and i
# We stop before the end to allow for entry at i+1
for i in range(2, len(df) - 1):
    # Get colors of the last 3 candles (i-2, i-1, i)
    color_at_i_minus_2 = df['Candle_Color'].iloc[i-2]
    color_at_i_minus_1 = df['Candle_Color'].iloc[i-1]
    color_at_i = df['Candle_Color'].iloc[i]
    
    # Check if all 3 candles are exactly the same color
    # This means either all are GREEN (1,1,1) or all are RED (0,0,0)
    if (color_at_i_minus_2 == color_at_i_minus_1 == color_at_i):
        
        # We found 3 consecutive same-color candles!
        # According to your strategy:
        # - Candle at i-2 = 1st candle of pattern
        # - Candle at i-1 = 2nd candle of pattern (SL will be calculated from this)
        # - Candle at i = 3rd candle of pattern
        # - Entry = close of candle at i+1 (next candle after the pattern)
        
        if (i + 1) < len(df):  # Make sure we have a next candle for entry
            
            # Entry details
            entry_index = i + 1
            entry_date = df.index[entry_index]
            entry_price = df['Close'].iloc[entry_index]  # Entry at close of 3rd candle
            
            # Determine signal type based on color
            if color_at_i == 1:  # All three are GREEN
                signal_type = 'BUY'
                # SL = midpoint of 2nd green candle (at i-1)
                sl_high = df['High'].iloc[i-1]
                sl_low = df['Low'].iloc[i-1]
                sl_price = (sl_high + sl_low) / 2
                
            else:  # All three are RED (color_at_i == 0)
                signal_type = 'SELL'
                # SL = midpoint of 2nd red candle (at i-1)
                sl_high = df['High'].iloc[i-1]
                sl_low = df['Low'].iloc[i-1]
                sl_price = (sl_high + sl_low) / 2
            
            # Calculate Take Profit based on SL distance
            if signal_type == 'BUY':
                # For BUY: Entry is below TP, SL is below Entry
                sl_distance = entry_price - sl_price  # How far SL is below entry
                tp_distance = 2 * sl_distance  # TP is 2x the risk
                tp_price = entry_price + tp_distance  # TP = Entry + 2×risk
                
            else:  # SELL
                # For SELL: Entry is above TP, SL is above Entry
                sl_distance = sl_price - entry_price  # How far SL is above entry
                tp_distance = 2 * sl_distance  # TP is 2x the risk
                tp_price = entry_price - tp_distance  # TP = Entry - 2×risk
            
            # Store this signal
            signals.append({
                'Entry_Date': entry_date,
                'Entry_Index': entry_index,
                'Pattern_Candle_1_Index': i-2,
                'Pattern_Candle_2_Index': i-1,
                'Pattern_Candle_3_Index': i,
                'Signal_Type': signal_type,
                'Entry_Price': entry_price,
                'SL_Price': sl_price,
                'SL_Distance': abs(sl_distance),
                'TP_Price': tp_price,
                'TP_Distance': tp_distance
            })

print(f"\n✓ Pattern Recognition Complete")
print(f"  Total patterns found: {len(signals)}")
print(f"  Data range: {df.index[0]} to {df.index[-1]}")

# Now simulate each signal as a trade
# For each signal, we look forward to see if SL or TP is hit first
trades = []

for signal in signals:
    entry_index = signal['Entry_Index']
    signal_type = signal['Signal_Type']
    entry_price = signal['Entry_Price']
    sl_price = signal['SL_Price']
    tp_price = signal['TP_Price']
    entry_date = signal['Entry_Date']
    
    # Trade state
    exit_price = None
    exit_reason = None
    bars_held = 0
    
    # Scan forward from entry to find where SL or TP is hit
    # We limit the search to 1440 candles (24 hours of 1-minute data)
    max_bars_to_check = 1440
    
    for j in range(entry_index + 1, min(entry_index + 1 + max_bars_to_check, len(df))):
        current_high = df['High'].iloc[j]
        current_low = df['Low'].iloc[j]
        bars_held = j - entry_index
        
        if signal_type == 'BUY':
            # For BUY: We're long, looking for price to go UP to TP or DOWN to SL
            # SL is below entry, TP is above entry
            
            # Check if SL is hit (price drops to or below SL)
            if current_low <= sl_price:
                # SL hit - we lose money
                exit_price = sl_price
                exit_reason = 'SL'
                break
            
            # Check if TP is hit (price rises to or above TP)
            elif current_high >= tp_price:
                # TP hit - we make money
                exit_price = tp_price
                exit_reason = 'TP'
                break
        
        else:  # SELL
            # For SELL: We're short, looking for price to go DOWN to TP or UP to SL
            # SL is above entry, TP is below entry
            
            # Check if SL is hit (price rises to or above SL)
            if current_high >= sl_price:
                # SL hit - we lose money
                exit_price = sl_price
                exit_reason = 'SL'
                break
            
            # Check if TP is hit (price falls to or below TP)
            elif current_low <= tp_price:
                # TP hit - we make money
                exit_price = tp_price
                exit_reason = 'TP'
                break
    
    # If we didn't hit SL or TP within 24 hours, trade is still open
    # We'll close it at the last available price
    if exit_reason is None:
        exit_reason = 'TIMEOUT'
        exit_price = df['Close'].iloc[-1]
        # Don't count timeouts as real trades (they're incomplete)
    
    # Calculate profit/loss
    if signal_type == 'BUY':
        pnl = exit_price - entry_price
        pnl_pct = (pnl / entry_price) * 100
    else:  # SELL
        pnl = entry_price - exit_price
        pnl_pct = (pnl / entry_price) * 100
    
    # Record the trade
    trades.append({
        'Entry_Date': entry_date,
        'Signal_Type': signal_type,
        'Entry_Price': entry_price,
        'SL_Price': sl_price,
        'TP_Price': tp_price,
        'Exit_Price': exit_price,
        'Exit_Reason': exit_reason,
        'PnL': pnl,
        'PnL_Pct': pnl_pct,
        'Bars_Held': bars_held,
        'Is_Win': pnl > 0,
        'Is_Completed': exit_reason in ['SL', 'TP']
    })

# Filter to only completed trades (those that hit SL or TP)
completed_trades = [t for t in trades if t['Is_Completed']]
trades_df = pd.DataFrame(completed_trades)

print(f"\n✓ Trade Simulation Complete")
print(f"  Total signals: {len(signals)}")
print(f"  Total trades executed: {len(trades)}")
print(f"  Completed trades (hit SL/TP): {len(trades_df)}")
print(f"  Incomplete trades (timeout): {len(trades) - len(trades_df)}")

# Calculate statistics from completed trades only
if len(trades_df) > 0:
    winning_trades = trades_df['Is_Win'].sum()
    losing_trades = (~trades_df['Is_Win']).sum()
    win_rate = (winning_trades / len(trades_df)) * 100
    
    avg_win = trades_df[trades_df['Is_Win']]['PnL'].mean()
    avg_loss = trades_df[~trades_df['Is_Win']]['PnL'].mean()
    
    total_pnl = trades_df['PnL'].sum()
    gross_profit = trades_df[trades_df['Is_Win']]['PnL'].sum()
    gross_loss = abs(trades_df[~trades_df['Is_Win']]['PnL'].sum())
    
    profit_factor = gross_profit / gross_loss if gross_loss > 0 else 0
    
    avg_bars_held = trades_df['Bars_Held'].mean()
    
    # Sharpe ratio (simplified): return / volatility
    returns = trades_df['PnL_Pct']
    sharpe = (returns.mean() / returns.std()) * np.sqrt(252) if returns.std() > 0 else 0
    
    print(f"\n" + "="*80)
    print(f"BASELINE STRATEGY RESULTS (YOUR 3-CANDLE PATTERN)")
    print(f"="*80)
    print(f"\nTrade Count:")
    print(f"  Total Completed Trades: {len(trades_df)}")
    print(f"  Winning Trades: {winning_trades} ({win_rate:.2f}%)")
    print(f"  Losing Trades: {losing_trades} ({100-win_rate:.2f}%)")
    
    print(f"\nProfitability:")
    print(f"  Total P&L: ${total_pnl:.4f}")
    print(f"  Gross Profit: ${gross_profit:.4f}")
    print(f"  Gross Loss: ${gross_loss:.4f}")
    print(f"  Profit Factor: {profit_factor:.2f} (>1.2 is good)")
    
    print(f"\nAverage Trade:")
    print(f"  Avg Win: ${avg_win:.4f}")
    print(f"  Avg Loss: ${avg_loss:.4f}")
    print(f"  Win/Loss Ratio: {avg_win/abs(avg_loss) if avg_loss != 0 else 0:.2f}")
    print(f"  Avg Bars Held: {avg_bars_held:.0f} minutes")
    
    print(f"\nRisk-Adjusted:")
    print(f"  Sharpe Ratio: {sharpe:.4f}")
    print(f"  Return per Trade (avg): {returns.mean():.6f}%")
    
    print(f"\n" + "="*80)
    print(f"STRATEGY EDGE ASSESSMENT:")
    print(f"="*80)
    
    # Assessment based on metrics
    if win_rate >= 40:
        print(f"✓ Win rate {win_rate:.2f}% >= 40% threshold (profitable with 1:2 risk-reward)")
    else:
        print(f"✗ Win rate {win_rate:.2f}% < 40% threshold (marginal edge)")
    
    if profit_factor >= 1.2:
        print(f"✓ Profit factor {profit_factor:.2f} >= 1.2 (good)")
    else:
        print(f"✗ Profit factor {profit_factor:.2f} < 1.2 (weak)")
    
    if sharpe >= 0.7:
        print(f"✓ Sharpe ratio {sharpe:.4f} >= 0.7 (acceptable)")
    else:
        print(f"✗ Sharpe ratio {sharpe:.4f} < 0.7 (poor risk-adjusted returns)")
    
    # Overall assessment
    edge_score = 0
    if win_rate >= 40: edge_score += 1
    if profit_factor >= 1.2: edge_score += 1
    if sharpe >= 0.7: edge_score += 1
    
    print(f"\nEdge Score: {edge_score}/3")
    if edge_score >= 2:
        print(f"→ EDGE EXISTS: Strategy shows statistically significant advantage")
        print(f"→ ML MODEL should aim to beat this baseline")
    else:
        print(f"→ MARGINAL EDGE: Strategy is borderline, needs careful tuning")

# ================================================================================
# SECTION 16: VISUALIZATIONS
# ================================================================================
# Now we create visualizations to see patterns visually

print("\n[SECTION 16] CREATING VISUALIZATIONS...")

# Visualization 1: Full price history with crisis periods marked
fig, ax = plt.subplots(figsize=(16, 7))

ax.plot(df.index, df['Close'], linewidth=0.5, color='steelblue', alpha=0.7)
ax.fill_between(df.index, df['Low'], df['High'], alpha=0.15, color='steelblue')

# Mark crisis periods
colors_crisis = {'2008 Financial Crisis': 'red', '2020 COVID-19 Crash': 'orange', '2022 Fed Rate Hikes': 'purple'}
for crisis_name, (start_date, end_date) in crises.items():
    ax.axvspan(start_date, end_date, alpha=0.1, color=colors_crisis[crisis_name], label=crisis_name)

ax.set_title('XAUUSD Price History (2004-2024) with Crisis Periods', fontsize=14, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Price (USD)', fontsize=12)
ax.legend(loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('01_price_history_with_crises.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 01_price_history_with_crises.png")
plt.close()

# Visualization 2: Returns distribution (histogram)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Simple returns
axes[0].hist(df['Simple_Return'].dropna() * 100, bins=100, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].axvline(df['Simple_Return'].mean() * 100, color='red', linestyle='--', linewidth=2, label=f'Mean: {df["Simple_Return"].mean()*100:.4f}%')
axes[0].set_title('Simple Returns Distribution', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Return (%)', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Log returns
axes[1].hist(df['Log_Return'].dropna() * 100, bins=100, color='darkgreen', alpha=0.7, edgecolor='black')
axes[1].axvline(df['Log_Return'].mean() * 100, color='red', linestyle='--', linewidth=2, label=f'Mean: {df["Log_Return"].mean()*100:.4f}%')
axes[1].set_title('Log Returns Distribution', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Return (%)', fontsize=11)
axes[1].set_ylabel('Frequency', fontsize=11)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('02_returns_distribution.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 02_returns_distribution.png")
plt.close()

# Visualization 3: Volatility over time
fig, ax = plt.subplots(figsize=(16, 6))

ax.plot(df.index, df['Volatility_1d'], linewidth=1, color='purple', alpha=0.7, label='1-Day Rolling Volatility')
ax.fill_between(df.index, df['Volatility_1d'], alpha=0.2, color='purple')

# Mark crisis periods
for crisis_name, (start_date, end_date) in crises.items():
    ax.axvspan(start_date, end_date, alpha=0.1, color=colors_crisis[crisis_name])

ax.set_title('XAUUSD Rolling Volatility (1-Day Window)', fontsize=14, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Daily Volatility (Std Dev)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('03_volatility_over_time.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 03_volatility_over_time.png")
plt.close()

# Visualization 4: Drawdown over time
fig, ax = plt.subplots(figsize=(16, 6))

cumulative_max = df['Close'].cummax()
drawdown = ((df['Close'] - cumulative_max) / cumulative_max) * 100

ax.fill_between(df.index, drawdown, 0, color='red', alpha=0.5, label='Drawdown')
ax.plot(df.index, drawdown, color='darkred', linewidth=0.8)

ax.set_title('XAUUSD Drawdown Over Time', fontsize=14, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Drawdown (%)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('04_drawdown_over_time.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 04_drawdown_over_time.png")
plt.close()

# Visualization 5: Candle colors (green vs red) over time
fig, ax = plt.subplots(figsize=(16, 5))

# Create a color array: green for up days, red for down days
colors = ['green' if color == 1 else 'red' for color in df['Candle_Color']]

# Show last 5000 candles for visibility
last_n = 5000
x_vals = range(len(df) - last_n, len(df))
y_vals = df['Close'].iloc[-last_n:].values

for i, (x, y, color) in enumerate(zip(x_vals, y_vals, colors[-last_n:])):
    ax.scatter(x, y, c=color, s=1, alpha=0.6)

ax.set_title(f'XAUUSD Candle Colors (Last {last_n} Candles)', fontsize=14, fontweight='bold')
ax.set_xlabel('Candle Index', fontsize=12)
ax.set_ylabel('Close Price', fontsize=12)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('05_candle_colors_last_5000.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 05_candle_colors_last_5000.png")
plt.close()

# ================================================================================
# SECTION 17: BIVARIATE ANALYSIS - OHLC RELATIONSHIPS
# ================================================================================
# How do Open, High, Low, Close relate to each other?

print("\n[SECTION 17] BIVARIATE ANALYSIS - OHLC RELATIONSHIPS...")

# Check: Does Close tend to be near High or Low?
df['Close_Position'] = (df['Close'] - df['Low']) / (df['High'] - df['Low'])

print(f"\nClose Position within Daily Range:")
print(f"  Mean: {df['Close_Position'].mean():.4f}")
print(f"  (0 = Close at Low, 1 = Close at High, 0.5 = Middle)")

if df['Close_Position'].mean() > 0.5:
    print(f"  → Close typically above middle (bullish tendency)")
else:
    print(f"  → Close typically below middle (bearish tendency)")

# Correlation between OHLC
print(f"\nOHLC Correlation Matrix:")
ohlc_corr = df[['Open', 'High', 'Low', 'Close']].corr()
print(ohlc_corr)

# ================================================================================
# SECTION 18: SUMMARY REPORT
# ================================================================================
# Final summary of key findings

print("\n" + "="*80)
print("SUMMARY REPORT - GENERAL EDA FINDINGS")
print("="*80)

summary_findings = {
    'Data Period': f"{df.index[0].date()} to {df.index[-1].date()}",
    'Total Candles': f"{len(df):,}",
    'Timezone': detected_timezone,
    'Data Quality': 'Excellent',
    'Price Range': f"${df['Close'].min():.2f} - ${df['Close'].max():.2f}",
    'Average Daily Volatility': f"{df['Volatility_1d'].mean():.6f}",
    'Annualized Volatility': f"{annual_volatility*100:.2f}%",
    'Max Drawdown': f"{max_drawdown:.2f}%",
    'Green vs Red Candles': f"{(green_candles/total_candles*100):.2f}% up, {(red_candles/total_candles*100):.2f}% down",
    'Baseline Strategy Signals': f"{len(signals)}",
    'Baseline Win Rate': f"{win_rate:.2f}%" if len(trades) > 0 else "N/A",
    'Volume Type': 'Likely Tick Volume (limited predictive power)',
    'Key Finding': 'Data is clean, 2008/2020 crises visible, volatility regimes detectable',
}

for key, value in summary_findings.items():
    print(f"{key:.<50} {value}")

print("="*80)

print("\n✓ GENERAL EDA COMPLETE!")
print("\nNext Steps:")
print("  1. Review all visualizations (PNG files)")
print("  2. Document findings in EXPLORATION_LOG.md")
print("  3. Use insights for feature engineering")
print("  4. Build baseline model based on 3-candle strategy")
print("  5. Move to formal trading-system architecture")

In [ ]:
# ================================================================================
# SECTION 15: BASELINE STRATEGY ANALYSIS - YOUR 3-CANDLE REVERSAL
# ================================================================================
# YOUR EXACT STRATEGY (Final Version):
# BUY: 3 consecutive GREEN candles → Enter at close of 3rd → SL at midpoint of 2nd
# SELL: 3 consecutive RED candles → Enter at close of 3rd → SL at midpoint of 2nd
# TP: Always 2× the SL distance
# EXIT: Only at SL or TP (candle wicks ignored)
# REQUIREMENT: Exactly 3 consecutive same-color candles (no partial patterns)

print("\n[SECTION 15] BASELINE STRATEGY ANALYSIS - YOUR 3-CANDLE PATTERN...")
print("\nStrategy Rules (FINAL):")
print("  BUY:  3 GREEN candles → Enter @ close of 3rd → SL @ midpoint of 2nd")
print("  SELL: 3 RED candles → Enter @ close of 3rd → SL @ midpoint of 2nd")
print("  TP: Entry ± 2×(SL distance)")
print("  EXIT: Only at SL or TP")
print("  REQUIREMENT: Exactly 3 consecutive same-color candles")

# ================================================================================
# TRANSACTION COST CONFIGURATION
# ================================================================================
# Define transaction costs for different broker types
BROKER_COSTS = {
    'standard': 0.45,  # Standard broker: 3 pips spread + slippage
    'ecn': 0.20        # ECN broker (e.g., Exness Raw Spread): tight spread + commission
}

# Select broker type for analysis
# Change this to 'ecn' if you want to test with ECN broker costs
SELECTED_BROKER = 'standard'  # Options: 'standard' or 'ecn'
transaction_cost_per_trade = BROKER_COSTS[SELECTED_BROKER]

print(f"\n⚠ TRANSACTION COST CONFIGURATION:")
print(f"  Broker Type: {SELECTED_BROKER.upper()}")
print(f"  Cost per Round-Trip Trade: ${transaction_cost_per_trade:.2f}")
print(f"  (Change SELECTED_BROKER variable to 'ecn' for lower costs)")

# Create candle color column
# GREEN = 1 (Close > Open), RED = 0 (Close < Open)
df['Candle_Color'] = (df['Close'] > df['Open']).astype(int)

signals = []  # Store all entry signals found

# Loop through data looking for exactly 3 consecutive same-color candles
# We start at index 2 because we need candles at i-2, i-1, and i
# We stop before the end to allow for entry at i+1
for i in range(2, len(df) - 1):
    # Get colors of the last 3 candles (i-2, i-1, i)
    color_at_i_minus_2 = df['Candle_Color'].iloc[i-2]
    color_at_i_minus_1 = df['Candle_Color'].iloc[i-1]
    color_at_i = df['Candle_Color'].iloc[i]
    
    # Check if all 3 candles are exactly the same color
    # This means either all are GREEN (1,1,1) or all are RED (0,0,0)
    if (color_at_i_minus_2 == color_at_i_minus_1 == color_at_i):
        
        # We found 3 consecutive same-color candles!
        # According to your strategy:
        # - Candle at i-2 = 1st candle of pattern
        # - Candle at i-1 = 2nd candle of pattern (SL will be calculated from this)
        # - Candle at i = 3rd candle of pattern
        # - Entry = immediately at the CLOSE of the 3rd candle (i)
            
        # Entry details
        entry_index = i
        entry_date = df.index[entry_index]
        entry_price = df['Close'].iloc[entry_index]  # Entry at close of 3rd candle
        
        # Determine signal type based on color
        if color_at_i == 1:  # All three are GREEN
            signal_type = 'BUY'
            # SL = midpoint of 2nd green candle (at i-1)
            sl_high = df['High'].iloc[i-1]
            sl_low = df['Low'].iloc[i-1]
            sl_price = (sl_high + sl_low) / 2
                
        else:  # All three are RED (color_at_i == 0)
            signal_type = 'SELL'
            # SL = midpoint of 2nd red candle (at i-1)
            sl_high = df['High'].iloc[i-1]
            sl_low = df['Low'].iloc[i-1]
            sl_price = (sl_high + sl_low) / 2
            
        # Calculate Take Profit based on SL distance
        if signal_type == 'BUY':
            # For BUY: Entry is below TP, SL is below Entry
            sl_distance = entry_price - sl_price  # How far SL is below entry
            tp_distance = 2 * sl_distance  # TP is 2x the risk
            tp_price = entry_price + tp_distance  # TP = Entry + 2×risk
            
        else:  # SELL
            # For SELL: Entry is above TP, SL is above Entry
            sl_distance = sl_price - entry_price  # How far SL is above entry
            tp_distance = 2 * sl_distance  # TP is 2x the risk
            tp_price = entry_price - tp_distance  # TP = Entry - 2×risk
            
        # Store this signal
        signals.append({
            'Entry_Date': entry_date,
            'Entry_Index': entry_index,
            'Pattern_Candle_1_Index': i-2,
            'Pattern_Candle_2_Index': i-1,
            'Pattern_Candle_3_Index': i,
            'Signal_Type': signal_type,
            'Entry_Price': entry_price,
            'SL_Price': sl_price,
            'SL_Distance': abs(sl_distance),
            'TP_Price': tp_price,
            'TP_Distance': tp_distance
        })

print(f"\n✓ Pattern Recognition Complete")
print(f"  Total patterns found: {len(signals)}")
print(f"  Data range: {df.index[0]} to {df.index[-1]}")

# Now simulate each signal as a trade
# For each signal, we look forward to see if SL or TP is hit first
trades = []

for signal in signals:
    entry_index = signal['Entry_Index']
    signal_type = signal['Signal_Type']
    entry_price = signal['Entry_Price']
    sl_price = signal['SL_Price']
    tp_price = signal['TP_Price']
    entry_date = signal['Entry_Date']
    
    # Trade state
    exit_price = None
    exit_reason = None
    bars_held = 0
    
    # Scan forward from entry to find where SL or TP is hit
    # We limit the search to 1440 candles (24 hours of 1-minute data)
    max_bars_to_check = 1440
    
    for j in range(entry_index + 1, min(entry_index + 1 + max_bars_to_check, len(df))):
        current_high = df['High'].iloc[j]
        current_low = df['Low'].iloc[j]
        bars_held = j - entry_index
        
        if signal_type == 'BUY':
            # For BUY: We're long, looking for price to go UP to TP or DOWN to SL
            # SL is below entry, TP is above entry
            
            # Check if SL is hit (price drops to or below SL)
            if current_low <= sl_price:
                # SL hit - we lose money
                exit_price = sl_price
                exit_reason = 'SL'
                break
            
            # Check if TP is hit (price rises to or above TP)
            elif current_high >= tp_price:
                # TP hit - we make money
                exit_price = tp_price
                exit_reason = 'TP'
                break
        
        else:  # SELL
            # For SELL: We're short, looking for price to go DOWN to TP or UP to SL
            # SL is above entry, TP is below entry
            
            # Check if SL is hit (price rises to or above SL)
            if current_high >= sl_price:
                # SL hit - we lose money
                exit_price = sl_price
                exit_reason = 'SL'
                break
            
            # Check if TP is hit (price falls to or below TP)
            elif current_low <= tp_price:
                # TP hit - we make money
                exit_price = tp_price
                exit_reason = 'TP'
                break
    
    # If we didn't hit SL or TP within 24 hours, trade is still open
    # We'll close it at the last available price
    if exit_reason is None:
        exit_reason = 'TIMEOUT'
        exit_price = df['Close'].iloc[-1]
        # Don't count timeouts as real trades (they're incomplete)
    
    # Calculate profit/loss (GROSS - before costs)
    if signal_type == 'BUY':
        pnl_gross = exit_price - entry_price
        pnl_pct_gross = (pnl_gross / entry_price) * 100
    else:  # SELL
        pnl_gross = entry_price - exit_price
        pnl_pct_gross = (pnl_gross / entry_price) * 100
    
    # ============================================================
    # APPLY TRANSACTION COSTS
    # ============================================================
    # Deduct costs from gross P&L to get net P&L
    pnl_net = pnl_gross - transaction_cost_per_trade
    pnl_pct_net = (pnl_net / entry_price) * 100
    
    # Determine win/loss status AFTER costs (realistic)
    is_win_net = pnl_net > 0
    # ============================================================
    
    # Record the trade
    trades.append({
        'Entry_Date': entry_date,
        'Signal_Type': signal_type,
        'Entry_Price': entry_price,
        'SL_Price': sl_price,
        'TP_Price': tp_price,
        'Exit_Price': exit_price,
        'Exit_Reason': exit_reason,
        'PnL_Gross': pnl_gross,                    # P&L before costs
        'PnL_Net': pnl_net,                        # P&L after costs (REAL)
        'Transaction_Cost': transaction_cost_per_trade,
        'PnL_Pct_Gross': pnl_pct_gross,
        'PnL_Pct_Net': pnl_pct_net,                # Real percentage return
        'Bars_Held': bars_held,
        'Is_Win_Gross': pnl_gross > 0,             # Win before costs
        'Is_Win_Net': is_win_net,                  # Win after costs (REAL)
        'Is_Completed': exit_reason in ['SL', 'TP']
    })

# Filter to only completed trades (those that hit SL or TP)
completed_trades = [t for t in trades if t['Is_Completed']]
trades_df = pd.DataFrame(completed_trades)

print(f"\n✓ Trade Simulation Complete")
print(f"  Total signals: {len(signals)}")
print(f"  Total trades executed: {len(trades)}")
print(f"  Completed trades (hit SL/TP): {len(trades_df)}")
print(f"  Incomplete trades (timeout): {len(trades) - len(trades_df)}")

# ================================================================================
# CALCULATE STATISTICS - BEFORE AND AFTER COSTS
# ================================================================================
if len(trades_df) > 0:
    
    # ============================================================
    # BEFORE COSTS (Gross/Theoretical)
    # ============================================================
    winning_trades_gross = trades_df['Is_Win_Gross'].sum()
    losing_trades_gross = (~trades_df['Is_Win_Gross']).sum()
    win_rate_gross = (winning_trades_gross / len(trades_df)) * 100
    
    avg_win_gross = trades_df[trades_df['Is_Win_Gross']]['PnL_Gross'].mean()
    avg_loss_gross = trades_df[~trades_df['Is_Win_Gross']]['PnL_Gross'].mean()
    
    total_pnl_gross = trades_df['PnL_Gross'].sum()
    gross_profit_gross = trades_df[trades_df['Is_Win_Gross']]['PnL_Gross'].sum()
    gross_loss_gross = abs(trades_df[~trades_df['Is_Win_Gross']]['PnL_Gross'].sum())
    
    profit_factor_gross = gross_profit_gross / gross_loss_gross if gross_loss_gross > 0 else 0
    
    # ============================================================
    # AFTER COSTS (Net/Realistic) - THIS IS WHAT MATTERS!
    # ============================================================
    winning_trades_net = trades_df['Is_Win_Net'].sum()
    losing_trades_net = (~trades_df['Is_Win_Net']).sum()
    win_rate_net = (winning_trades_net / len(trades_df)) * 100
    
    avg_win_net = trades_df[trades_df['Is_Win_Net']]['PnL_Net'].mean()
    avg_loss_net = trades_df[~trades_df['Is_Win_Net']]['PnL_Net'].mean()
    
    total_pnl_net = trades_df['PnL_Net'].sum()
    gross_profit_net = trades_df[trades_df['Is_Win_Net']]['PnL_Net'].sum()
    gross_loss_net = abs(trades_df[~trades_df['Is_Win_Net']]['PnL_Net'].sum())
    
    profit_factor_net = gross_profit_net / gross_loss_net if gross_loss_net > 0 else 0
    
    # ============================================================
    # ADDITIONAL METRICS
    # ============================================================
    total_costs = trades_df['Transaction_Cost'].sum()
    avg_bars_held = trades_df['Bars_Held'].mean()
    
    # Sharpe ratio (using NET returns - realistic)
    returns_net = trades_df['PnL_Pct_Net']
    sharpe_net = (returns_net.mean() / returns_net.std()) * np.sqrt(252) if returns_net.std() > 0 else 0
    
    # Calculate breakeven win rate (after costs)
    breakeven_win_rate = abs(avg_loss_net) / (avg_win_net + abs(avg_loss_net)) * 100 if (avg_win_net + abs(avg_loss_net)) != 0 else 0
    edge_over_breakeven = win_rate_net - breakeven_win_rate
    
    # ============================================================
    # DISPLAY RESULTS
    # ============================================================
    print(f"\n" + "="*80)
    print(f"BASELINE STRATEGY RESULTS (YOUR 3-CANDLE PATTERN)")
    print(f"="*80)
    
    # --------------------------------------------------------
    # SECTION 1: BEFORE TRANSACTION COSTS (Theoretical)
    # --------------------------------------------------------
    print(f"\n" + "="*80)
    print(f"BEFORE TRANSACTION COSTS (Theoretical Performance)")
    print(f"="*80)
    print(f"\nTrade Count:")
    print(f"  Total Completed Trades: {len(trades_df):,}")
    print(f"  Winning Trades: {winning_trades_gross:,} ({win_rate_gross:.2f}%)")
    print(f"  Losing Trades: {losing_trades_gross:,} ({100-win_rate_gross:.2f}%)")
    
    print(f"\nProfitability (Gross):")
    print(f"  Total P&L: ${total_pnl_gross:.2f}")
    print(f"  Gross Profit: ${gross_profit_gross:.2f}")
    print(f"  Gross Loss: ${gross_loss_gross:.2f}")
    print(f"  Profit Factor: {profit_factor_gross:.2f}")
    
    print(f"\nAverage Trade (Gross):")
    print(f"  Avg Win: ${avg_win_gross:.4f}")
    print(f"  Avg Loss: ${avg_loss_gross:.4f}")
    print(f"  Win/Loss Ratio: {avg_win_gross/abs(avg_loss_gross) if avg_loss_gross != 0 else 0:.2f}")
    
    # --------------------------------------------------------
    # SECTION 2: AFTER TRANSACTION COSTS (Realistic) ← CRITICAL!
    # --------------------------------------------------------
    print(f"\n" + "="*80)
    print(f"AFTER TRANSACTION COSTS (REALISTIC Performance) ← USE THIS!")
    print(f"="*80)
    print(f"\nTransaction Cost Configuration:")
    print(f"  Broker Type: {SELECTED_BROKER.upper()}")
    print(f"  Cost per Trade: ${transaction_cost_per_trade:.2f}")
    print(f"  Total Costs Paid: ${total_costs:.2f}")
    
    print(f"\nTrade Count:")
    print(f"  Total Completed Trades: {len(trades_df):,}")
    print(f"  Winning Trades: {winning_trades_net:,} ({win_rate_net:.2f}%)")
    print(f"  Losing Trades: {losing_trades_net:,} ({100-win_rate_net:.2f}%)")
    
    print(f"\nProfitability (Net):")
    print(f"  Total P&L: ${total_pnl_net:.2f}")
    print(f"  Gross Profit: ${gross_profit_net:.2f}")
    print(f"  Gross Loss: ${gross_loss_net:.2f}")
    print(f"  Profit Factor: {profit_factor_net:.2f} (>1.2 is good)")
    
    print(f"\nAverage Trade (Net):")
    print(f"  Avg Win: ${avg_win_net:.4f}")
    print(f"  Avg Loss: ${avg_loss_net:.4f}")
    print(f"  Win/Loss Ratio: {avg_win_net/abs(avg_loss_net) if avg_loss_net != 0 else 0:.2f}")
    print(f"  Avg Bars Held: {avg_bars_held:.0f} minutes")
    
    print(f"\nRisk-Adjusted:")
    print(f"  Sharpe Ratio: {sharpe_net:.4f}")
    print(f"  Return per Trade (avg): {returns_net.mean():.6f}%")
    
    print(f"\nPer 100 Trades:")
    print(f"  Net Profit: ${(total_pnl_net/len(trades_df))*100:.2f}")
    print(f"  Total Costs: ${(total_costs/len(trades_df))*100:.2f}")
    
    # --------------------------------------------------------
    # SECTION 3: COST IMPACT ANALYSIS
    # --------------------------------------------------------
    print(f"\n" + "="*80)
    print(f"COST IMPACT ANALYSIS")
    print(f"="*80)
    
    profit_lost_to_costs = total_pnl_gross - total_pnl_net
    profit_lost_pct = (profit_lost_to_costs / total_pnl_gross * 100) if total_pnl_gross > 0 else 0
    
    print(f"\nProfit Erosion:")
    print(f"  Profit Before Costs: ${total_pnl_gross:.2f}")
    print(f"  Profit After Costs: ${total_pnl_net:.2f}")
    print(f"  Lost to Costs: ${profit_lost_to_costs:.2f} ({profit_lost_pct:.1f}%)")
    
    win_rate_change = win_rate_gross - win_rate_net
    trades_flipped = abs(winning_trades_gross - winning_trades_net)
    
    print(f"\nWin Rate Impact:")
    print(f"  Before Costs: {win_rate_gross:.2f}%")
    print(f"  After Costs: {win_rate_net:.2f}%")
    print(f"  Change: {win_rate_change:+.2f}%")
    print(f"  Trades Flipped to Losses: {trades_flipped} ({trades_flipped/len(trades_df)*100:.1f}%)")
    
    print(f"\nEdge Analysis:")
    print(f"  Breakeven Win Rate (after costs): {breakeven_win_rate:.2f}%")
    print(f"  Your Win Rate: {win_rate_net:.2f}%")
    print(f"  Edge Over Breakeven: {edge_over_breakeven:+.2f}%")
    
    if edge_over_breakeven > 0:
        print(f"  → POSITIVE EDGE EXISTS ✓")
    else:
        print(f"  → NO EDGE (strategy loses money after costs) ✗")
    
    # --------------------------------------------------------
    # SECTION 4: BROKER COMPARISON
    # --------------------------------------------------------
    print(f"\n" + "="*80)
    print(f"BROKER COMPARISON (What-If Analysis)")
    print(f"="*80)
    
    # Calculate what results would be with different broker
    if SELECTED_BROKER == 'standard':
        alt_broker = 'ecn'
        alt_cost = BROKER_COSTS['ecn']
    else:
        alt_broker = 'standard'
        alt_cost = BROKER_COSTS['standard']
    
    # Estimate performance with alternative broker
    cost_difference = transaction_cost_per_trade - alt_cost
    alt_total_pnl = total_pnl_net + (cost_difference * len(trades_df))
    alt_avg_win = avg_win_net + cost_difference if winning_trades_net > 0 else 0
    alt_avg_loss = avg_loss_net - cost_difference if losing_trades_net > 0 else 0
    
    # Recalculate win rate with alt broker (some losses might become wins)
    potential_flips = int(abs(cost_difference / (avg_win_gross + abs(avg_loss_gross)) * len(trades_df)))
    alt_winning_trades = min(winning_trades_net + potential_flips, len(trades_df))
    alt_win_rate = (alt_winning_trades / len(trades_df)) * 100
    
    improvement_pct = ((alt_total_pnl - total_pnl_net) / abs(total_pnl_net) * 100) if total_pnl_net != 0 else 0
    
    print(f"\nCurrent Broker ({SELECTED_BROKER.upper()}):")
    print(f"  Cost per Trade: ${transaction_cost_per_trade:.2f}")
    print(f"  Win Rate: {win_rate_net:.2f}%")
    print(f"  Total P&L: ${total_pnl_net:.2f}")
    print(f"  Profit per 100 Trades: ${(total_pnl_net/len(trades_df))*100:.2f}")
    
    print(f"\nAlternative Broker ({alt_broker.upper()}):")
    print(f"  Cost per Trade: ${alt_cost:.2f}")
    print(f"  Estimated Win Rate: ~{alt_win_rate:.2f}%")
    print(f"  Estimated Total P&L: ${alt_total_pnl:.2f}")
    print(f"  Estimated Profit per 100 Trades: ${(alt_total_pnl/len(trades_df))*100:.2f}")
    
    print(f"\nSavings with {alt_broker.upper()} Broker:")
    print(f"  Cost Savings per Trade: ${abs(cost_difference):.2f}")
    print(f"  Total Savings: ${abs(cost_difference) * len(trades_df):.2f}")
    print(f"  Profit Improvement: {improvement_pct:+.1f}%")
    
    if alt_total_pnl > total_pnl_net:
        print(f"  → Recommendation: Switch to {alt_broker.upper()} broker ✓")
    else:
        print(f"  → Current broker is better ✓")
    
    # --------------------------------------------------------
    # SECTION 5: STRATEGY EDGE ASSESSMENT
    # --------------------------------------------------------
    print(f"\n" + "="*80)
    print(f"STRATEGY EDGE ASSESSMENT (Based on Net Performance)")
    print(f"="*80)
    
    # Assessment based on NET metrics (after costs)
    edge_checks = []
    
    if win_rate_net >= 40:
        print(f"✓ Win rate {win_rate_net:.2f}% >= 40% threshold (profitable with 1:2 risk-reward)")
        edge_checks.append(True)
    else:
        print(f"✗ Win rate {win_rate_net:.2f}% < 40% threshold (marginal edge)")
        edge_checks.append(False)
    
    if profit_factor_net >= 1.2:
        print(f"✓ Profit factor {profit_factor_net:.2f} >= 1.2 (good)")
        edge_checks.append(True)
    else:
        print(f"✗ Profit factor {profit_factor_net:.2f} < 1.2 (weak)")
        edge_checks.append(False)
    
    if sharpe_net >= 0.7:
        print(f"✓ Sharpe ratio {sharpe_net:.4f} >= 0.7 (acceptable)")
        edge_checks.append(True)
    else:
        print(f"✗ Sharpe ratio {sharpe_net:.4f} < 0.7 (poor risk-adjusted returns)")
        edge_checks.append(False)
    
    if edge_over_breakeven > 2.0:
        print(f"✓ Edge over breakeven {edge_over_breakeven:.2f}% > 2% (good cushion)")
        edge_checks.append(True)
    else:
        print(f"⚠ Edge over breakeven {edge_over_breakeven:.2f}% < 2% (tight margin)")
        edge_checks.append(False)
    
    # Overall assessment
    edge_score = sum(edge_checks)
    
    print(f"\nEdge Score: {edge_score}/4")
    
    if edge_score >= 3:
        print(f"→ STRONG EDGE: Strategy shows statistically significant advantage ✓")
        print(f"→ ML MODEL should aim to beat this baseline")
        print(f"→ Strategy is READY for further development")
    elif edge_score >= 2:
        print(f"→ MODERATE EDGE: Strategy is profitable but needs optimization ⚠")
        print(f"→ Consider: ECN broker, volatility filters, or better entry timing")
        print(f"→ ML MODEL has room for improvement")
    else:
        print(f"→ WEAK/NO EDGE: Strategy is marginal after costs ✗")
        print(f"→ Recommendation: Major changes needed before proceeding")
        print(f"→ Consider: Different pattern, timeframe, or instrument")
    
    print(f"\n" + "="*80)
    print(f"⚠ CRITICAL: Always use NET (after-cost) performance for decisions!")
    print(f"⚠ Gross performance is misleading and will cause losses in live trading!")
    print(f"="*80)

### Transaction Costs: Included
```
BROKER_COSTS = {
    'standard': 0.45,  # ← This IS transaction cost
    'ecn': 0.20        # ← This IS transaction cost
}
```

### Spread: Included (part of transaction cost)
```
'standard': 0.45  # = $0.30 spread + $0.15 slippage
'ecn': 0.20       # = $0.05 spread + $0.15 slippage
```


### Slippage: Include (part of transaction cost)
- The $0.45 or $0.20 already contains slippage estimate
- From Section 14:
- Standard broker: $0.30 spread + $0.15 slippage = $0.45
- ECN broker: $0.05 spread + $0.15 slippage = $0.20
```

---

## Breakdown of What Each Cost Represents:

### **Standard Broker ($0.45 total):**
```
Entry:
- Spread cost: $0.30 (you pay 3 pips to enter)

Exit:
- Spread cost: $0.00 (already included in entry)
- Slippage: $0.15 (execution delay, price moves)

Total: $0.45 per round-trip trade
```

### **ECN Broker ($0.20 total):**
```
Entry:
- Spread cost: $0.05 (you pay 0.5 pips to enter)
- Commission: $0.035 (fixed fee per 0.01 lot)

Exit:
- Spread cost: $0.00 (minimal)
- Commission: $0.035 (fixed fee per 0.01 lot)
- Slippage: $0.10 (better execution, less slippage)

Total: $0.20 per round-trip trade


### What this means:
- When you see this in output:
`pnl_net = pnl_gross - transaction_cost_per_trade`

- It means:
`pnl_net = raw_profit - (spread + slippage + commission)`

**Example:**

- Raw profit: $2.00
- Costs: $0.45
- Net profit: $2.00 - $0.45 = $1.55 ← THIS is what you ACTUALLY make